# Extract MoBiAS PDF reports

The PDF reports are a great source of additional information as they contain data for peaks that may not have been assigned to any known product.
Mining these reports is much faster than reprocessing of the raw LCMS data.

Here, we are in particular interested to identify
1. leftover starting materials
2. systematic side products that we have not been previously looking for


In [1]:
import sys
import pathlib
import re

sys.path.insert(0, str(pathlib.Path().resolve().parents[1]))

from pypdf import PdfReader
import pandas as pd
import numpy as np

from src.util.db_utils import SynFermDatabaseConnection
from src.definitions import DATA_DIR, PLATE_LIST_PATH
from src.util.rdkit_util import smiles_to_lcms_mass

## Extract PDF data

In [2]:
con = SynFermDatabaseConnection()

In [5]:
list(con.get_product_mol(3085, "ABC"))

In [14]:
def import_lcms_full_report(path):
    # set up pdf reader
    reader = PdfReader(path)
    number_of_pages = len(reader.pages)
    i = 1
    data_page = True
    lines = []
    # iterate from second page until entire peak summary table is read
    for i in range(1, number_of_pages):
        page = reader.pages[i]
        text = page.extract_text()
        line_list = text.splitlines()
        if line_list[0] != '# RT [min] Area I S/N Max. m/z FWHM [min] Area % Int. %':  # stop when not encountering another data header
            break
        lines += line_list
    
    if len(lines) == 0:
        raise RuntimeError("No data extracted")
        
    # remove spaces in header
    cleaned_lines = ['# RT[min] Area I S/N max_m/z FWHM[min] Area% Int%',]
    # remove footers and data headers on not-first page
    cleaned_lines += [line for line in lines if line[0].isnumeric()] 

    # split the lines into individual fields
    data = [line.split() for line in cleaned_lines]
    
    # assemble DataFrame from data
    df = pd.DataFrame(data[1:], columns=data[0]).astype("float")

    return df

In [27]:
# import the plate list to obtain LCMS identifier - plate_nr relation
plate_list = pd.read_csv(PLATE_LIST_PATH)

for exp_nr in range(1, 30):
    for plate_nr in range(1, 7):
        lcms_id = plate_list.loc[(plate_list["exp_nr"] == exp_nr) & (plate_list["plate_nr"] == plate_nr), "results_file_name"].item().split("_")[0]
        exp_path = DATA_DIR / "tmp_pdf_reports" / lcms_id
        full_report_paths = list(exp_path.glob("*_LCMS_Fullreport.pdf"))
        
        for path in full_report_paths:
            print(lcms_id)
            # get well from filename
            regex = r'_P\d{1}-[A-Z]-\d{1,2}_'
            match = re.search(regex, path.name)
            well = "".join(match.group().strip("_").split("-")[1:])
            reaction_id = con.get_reaction_id((exp_nr, plate_nr, well))
            print(well)
            print(reaction_id)
            df = import_lcms_full_report(path)
            # we persist this to the database for re-use
            # first reformat df to fit DB
            df.insert(0, "reaction_id", reaction_id)
            df.insert(2, "retention_time_s", (df["RT[min]"] * 60).astype("int"))
            df = df.drop(columns="RT[min]")
            df = df.rename(columns={"#": "peak_number", 
                       "Area": "area", 
                       "I": "intensity",
                       "S/N": "signal_to_noise",
                       "max_m/z": "mz_max",
                       "FWHM[min]": "fwhm_min", 
                       "Area%": "%area",
                       "Int%": "%intensity"
                      }).astype({"peak_number": "int",
                                 "area": "int",
                                 "intensity": "int"})
            # write all extracted peaks to DB
            with con.con:
                con.con.executemany(
                    'INSERT INTO lcms_peaks (experiment_id, peak_nr, retention_time_s, area, intensity, signal_to_noise, mz_max, fwhm_min, "%area", "%intensity") VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?);', 
                    [tuple(row) for row in df.to_numpy()]
                )

BMII002049
K7
3085
BMII002049
L21
3119
BMII002233
A4
53216
BMII002233
A6
53218


In [6]:
# now which of these are unexpected / not already explained?
mz_dmso = [79.0212, 101.0032, 157.0351]  # M+H+, M+Na+ 2M+H+
mz_lock = [142.1590, 322.0481]  # tetramethylpiperidine, hexamethoxyphosphazene
mz_is = 361.1201  # fenofibrate

known_product_smiles = con.get_product_smiles((exp_nr, plate_nr), "A3")
known_product_mzs = [smiles_to_lcms_mass(smi) for smi in known_product_smiles]

known_mzs = mz_dmso + mz_lock + [mz_is] + known_product_mzs

In [7]:
unexplained_peaks = df.loc[((df["Area%"] > 5)  # only significant peaks
        & ~np.isclose(df["max_m/z"].to_numpy()[:, None], known_mzs, rtol=5e-6, atol=0).any(axis=1)  # only peaks that are not already explained
       ), 
    ]
unexplained_peaks

,RT[min],Area,I,S/N,max_m/z,FWHM[min],Area%,Int%
#,,,,,,,,
9,2.156,2.589547e+05,62241.0,17.8,146.1175,0.059,5.39209,3.71
12,2.903,6.490781e+05,146339.0,42.1,365.1712,0.081,13.51545,8.73
16,3.000,1.114304e+06,440712.0,130.7,365.1709,0.038,23.20264,26.28
17,3.395,2.613391e+06,1012323.0,306.9,332.0521,0.037,54.41742,60.36
22,3.592,1.006776e+06,445752.0,132.9,459.1523,0.034,20.96362,26.58
25,3.730,8.154621e+05,335406.0,100.8,505.1579,0.037,16.97999,20.00
36,5.032,4.152705e+05,143717.0,1209.5,250.9768,0.043,8.64698,8.57
37,5.033,3.906193e+05,139087.0,41.9,250.9768,0.043,8.13368,8.29


## Identify starting material peaks

In [9]:
sms = con.get_starting_materials_for_reaction((exp_nr, plate_nr, "K7"))
sms

['O=C(c1ccc(C2=NCCO2)cc1)[B-](F)(F)F.[K+]',
 'CC(C)C[C@H]1C[C@]2(O[NH2+]1)OC1(CCCCC1)OC2=O.[Cl-]',
 '[Cl-].[NH3+]NC(=S)c1cccs1']

In [12]:
smiles_to_lcms_mass(sms[2])

159.00451664409

In [13]:
unexplained_peaks.columns

Index(['RT[min]', 'Area', 'I', 'S/N', 'max_m/z', 'FWHM[min]', 'Area%', 'Int%'], dtype='object')